In [ ]:
import time
import json

# Get tokens list from Coin Market Cap


In [1]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/map'
parameters = {
  'start':'1',
  'limit':'5000',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': 'd0846215-0c0f-4fcb-82a7-7755f505c9d3',
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url)
  dataCMC = json.loads(response.text)

  print(dataCMC['data'][1002])
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

{'id': 3662, 'name': 'HedgeTrade', 'symbol': 'HEDG', 'slug': 'hedgetrade', 'rank': 399, 'displayTV': 1, 'is_active': 1, 'first_historical_data': '2019-01-03T17:49:38.000Z', 'last_historical_data': '2022-11-02T07:59:00.000Z', 'platform': {'id': 1027, 'name': 'Ethereum', 'symbol': 'ETH', 'slug': 'ethereum', 'token_address': '0xF1290473E210b2108A85237fbCd7b6eb42Cc654F'}}


## Remove Tokens which platform == None

In [2]:
ids = []
for coin_data in dataCMC['data']:
  ids.append(coin_data['id'])

[{'slug': 'belacoin',
  'address': '0x2e98a6804e4b6c832ed0ca876a943abd3400b224',
  'chain': 'ETH',
  'id': 217},
 {'slug': 'gamecredits',
  'address': '0x63f88a2298a5c4aee3c216aa6d926b184a4b2437',
  'chain': 'ETH',
  'id': 576},
 {'slug': 'circuits-of-value',
  'address': '0x3d658390460295fb963f54dc0899cfb1c30776df',
  'chain': 'ETH',
  'id': 788},
 {'slug': 'tether',
  'address': '0xdac17f958d2ee523a2206206994597c13d831ec7',
  'chain': 'ETH',
  'id': 825},
 {'slug': 'xaurum',
  'address': '0x4DF812F6064def1e5e029f1ca858777CC98D2D81',
  'chain': 'ETH',
  'id': 895},
 {'slug': 'agoras-tokens',
  'address': '0x738865301a9b7dd80dc3666dd48cf034ec42bdda',
  'chain': 'ETH',
  'id': 1037},
 {'slug': 'augur',
  'address': '0x1985365e9f78359a9B6AD760e32412f4a445E862',
  'chain': 'ETH',
  'id': 1104},
 {'slug': 'digixdao',
  'address': '0xe0b7927c4af23765cb51314a0e0521a9645f0e2a',
  'chain': 'ETH',
  'id': 1229},
 {'slug': 'pluton',
  'address': '0xd8912c10681d8b21fd3742244f44658dba12264e',
  'c

## Get tokens data, batch_size = 100

In [ ]:
count = 0
_id = ''
cmc_data = []
for id in ids:
  count += 1
  if count % 100 == 0 or count == len(ids):
    time.sleep(3)
    _id = _id + str(id)
    url = f'https://pro-api.coinmarketcap.com/v2/cryptocurrency/info?id={_id}'.format(_id)
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': 'd0846215-0c0f-4fcb-82a7-7755f505c9d3',
    }

    session = Session()
    session.headers.update(headers)

    try:
      response = session.get(url)
      fetched_data = json.loads(response.text)
      for coin in fetched_data['data']:
        coin_data = fetched_data['data'][coin]
        cmc_data.append({
          'slug': coin_data['slug'],
          'contract_address': coin_data['contract_address']
        })
      # print(cmc_data)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
      print(e)
    _id =''
  else:
    _id = _id + str(id) +  ','

## Store tokens data

In [3]:
with open('data.json', 'w') as outfile:
      json.dump(cmc_data, outfile, indent=4)

NameError: name 'cmc_data' is not defined

## Split data into 4 categories based on platform

In [4]:
data_BNB = []
data_ETH = []
data_FTM = []
data_MATIC = []
for coin in cmc_data:
  for pfs in coin['contract_address']:
    if pfs['platform']['coin']['symbol'] == 'BNB':
      data_BNB.append({
        'slug': coin['slug'],
        'address': pfs['contract_address']
      })
    if pfs['platform']['coin']['symbol'] == 'ETH' :
      data_ETH.append({
        'slug': coin['slug'],
        'address': pfs['contract_address']
      })
    if pfs['platform']['coin']['symbol'] == 'FTM':
      data_FTM.append({
        'slug': coin['slug'],
        'address': pfs['contract_address']
      })
    if pfs['platform']['coin']['symbol'] == 'MATIC':
      data_MATIC.append({
        'slug': coin['slug'],
        'address': pfs['contract_address']
      })
with open('data_BNB.json', 'w') as outfile:
      json.dump(data_BNB, outfile, indent=4)
with open('data_ETH.json', 'w') as outfile:
      json.dump(data_ETH, outfile, indent=4)
with open('data_FTM.json', 'w') as outfile:
      json.dump(data_FTM, outfile, indent=4)
with open('data_MATIC.json', 'w') as outfile:
      json.dump(data_MATIC, outfile, indent=4)

NameError: name 'cmc_data' is not defined

## Store all data into cmcdata.json

In [ ]:
with open('cmcdata.json', 'w') as outfile:
  json.dump(data_BNB + data_ETH + data_FTM + data_MATIC, outfile, indent=4)

# Main 

In [ ]:
f = open('origin/bsc_tokens.json')
data = json.load(f)

In [6]:
def augment_data(file_name, data):
  f = open(f'origin/{file_name}.json'.format(file_name))
  odata = json.load(f)
  print(len(odata))

  augmented_data = [] # Tokens which can find cmc_id
  unaugmented_data = [] # Tokens which cannot find cmc_id
  for odoc in odata:
    augmented = False
    for doc in data:
      if odoc['address'].lower() == doc['address'].lower():
        odoc['cmc_id'] = doc['slug']
        augmented = True 
        augmented_data.append(odoc)
    if not augmented:
      unaugmented_data.append(odoc) 
  print(len(augmented_data), len(unaugmented_data))
    
  with open(f'augmented/{file_name}.json'.format(file_name), 'w') as outfile:
    json.dump(augmented_data, outfile, indent=4)
  with open(f'unaugmented/{file_name}.json'.format(file_name), 'w') as outfile:
    json.dump(unaugmented_data, outfile, indent=4)

In [8]:
augment_data('bsc_tokens',data)
augment_data('eth_tokens', data)
augment_data('ftm_tokens', data)
augment_data('polygon_tokens', data)

1360
805 601
3232
1979 1314
83
54 34
23
13 11
